In [0]:
# --- Secure config for ADLS access ---
spark.conf.set(
    "fs.azure.account.key.goodreadsreviews60302712.dfs.core.windows.net",
    "00u5OPJ3GYKrSACO9uvWxs8ZikHiJ7FosLxuXngxN1GqYBs5f2SLr/V0yESwhmfXXSovGYabn6Ai+AStqwR5Tg=="
)


In [0]:
# Load curated reviews Delta table (Gold preparation start)
spark.sql("USE SCHEMA default")
curated = spark.table("default.curated_reviews")

print("Rows in curated dataset:", curated.count())
curated.printSchema()
display(curated.limit(5))


Rows in curated dataset: 14970953
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)



review_id,book_id,title,author_id,name,user_id,rating,review_text,language_code,n_votes,date_added
5601fa84af96135f04054eba75ab7d79,852470,"I'd Tell You I Love You, But Then I'd Have to Kill You (Gallagher Girls, #1)",56224,Ally Carter,5cca1dd30cd5a98c1c8e731839265ccf,5,"Tirzah from The Compulsive Reader let me borrow books 1-3 of the Gallagher Girl series, and I was so excited to have them. One of my favorite book settings is boarding school, and that's exactly where these books are set (yay!). The main premise of these books is that there is a school for girls who are training to be spies. This means plenty of cool classes and tasks (all spy related, of course). These kept my attention the most, because I found the author to be quite clever with her use of the spy techniques the girls were using. I really enjoyed this book and am glad I have the next few in the series to read right away - I want to know what else they learn about spying and more about the creative spy school that Ally Carter has created!",en-US,0,Sun Mar 21 18:05:41 -0700 2010
49c482951cbe28a271e56ffff88ecdeb,140082,"Club Dead (Sookie Stackhouse, #3)",17061,Charlaine Harris,5752e83b5c1ccdd8014aaf358b80c199,4,"Still loving these books and flying through them. With the risk of spoiling, I hope we get a full explanation of Bill's disappearance. I felt unfulfilled.",en-GB,0,Wed Aug 25 04:04:35 -0700 2010
fafe2b36e88f4e537b6f58a326beba89,15801668,The Girls of Atomic City: The Untold Story of the Women Who Helped Win World War II,166990,Denise Kiernan,230ca5b40e729a52912eb49b46670a0a,3,"This was an interesting book, but I would have liked more personal stories and less technical information. Also, since I didn't get to sit down and read this at one sitting, it was difficult to remember who some of the women were in the narrative. I can't imagine how much research went into this book, as it covered quite a broad range of topics. If you are interested in the making of the atomic bomb, this book would be a good reference book and the notes on the author's research would be very useful for further research.",eng,0,Wed Jul 10 13:22:25 -0700 2013
bbb57362ff984eba14f5fc54f33e2acc,13539044,The Silver Linings Playbook,1251730,Matthew Quick,792515885276e6d8087ceeef76f36610,4,"Book was good. As always, we were upset when the movie didn't follow. not that we were all that surprised.",eng,0,Fri Oct 04 22:51:24 -0700 2013
191e55b1300be469f75b4bd62af72974,6352093,"The Girl Who Kicked the Hornet's Nest (Millennium, #3)",706255,Stieg Larsson,dc71fd873b60d58d171a762b56daab9c,5,"Este livro foi uma montanha-russa de emocoes. Enquanto o primeiro livro tinha descricao a mais e o segundo livro tinha partes completamente desnecessarias, a mim parece-me que o Larsson encontrou o equilibrio perfeito com este terceiro e ultimo volume da trilogia. Este foi o meu favorito das 3 obras da trilogia Millenium e consegue condensar tudo o que Larsson tem de bom na sua escrita - as personagens incriveis; as complexas conspiracoes; os jogos de poder entre media e policia que enquanto estudante de comunicacao tanto aprecio. Com este livro Larsson introduziu uma side-story que tinha duvidas se seria alguma coisa de jeito. Resultado - foi tambem incrivel. Se em outros livros passava o tempo a pensar quantas paginas faltavam para acabar um capitulo, comecei a perder o habito com esta trilogia - o ritmo e as divisoes dentro de capitulos eram tao boas que nao havia qualquer necessidade disso. O sistema de perspetivas das diferentes personagens que criou e o melhor que vi ate hoje, superando ate o de George R. R. Martin. Em termos da accao deste livro e do desenvolvimento do enredo, a parte mais unica e curiosa e que o autor nunca nos poe numa posicao omnisciente, mas sim sempre do lado dos investigadores que a pouco e pouco vao desenrolando o intrincado novelo que constitui a trama. As personagens - as mais incriveis e realistas que li ate hoje. Eu acredito que haja por ai uma Salander e uma Berger e uma G

In [0]:
from pyspark.sql import functions as F

df = curated

# Normalize text: trim spaces, lowercase optional, remove control chars
def normalize_text(colname):
    return F.lower(
        F.regexp_replace(
            F.trim(F.col(colname)),
            r"[\x00-\x1F\x7F]", " "
        )
    )

df = (
    df
    .withColumn("review_text", normalize_text("review_text"))
    .withColumn("title", F.trim(F.col("title")))
    .withColumn("name", F.trim(F.col("name")))
    .filter(F.length("review_text") > 10)
)

# Sanity checks: rating range and nulls
df = df.filter((F.col("rating") >= 1) & (F.col("rating") <= 5))
df = df.dropna(subset=["review_id", "book_id", "author_id", "review_text"])
df_clean = df.dropDuplicates(["review_id"]).cache()

print("Rows after final cleaning:", df_clean.count())


Rows after final cleaning: 14926784


In [0]:
 # Compute review length (in words)
df_feat = df_clean.withColumn(
    "review_length_words",
    F.size(F.split(F.col("review_text"), r"\s+"))
)

# Aggregate by book_id
agg = (
    df_feat.groupBy("book_id")
    .agg(
        F.avg("rating").alias("avg_rating"),
        F.count("*").alias("num_reviews")
    )
)

# Join back to main DataFrame
features = (
    df_feat.join(agg, "book_id", "left")
)

display(features.limit(5))


book_id,review_id,title,author_id,name,user_id,rating,review_text,language_code,n_votes,date_added,review_length_words,avg_rating,num_reviews
21010103,00038a73f8d2751474db182acda480ae,"Push (Push, #1)",7942379,Claire Wallis,e146efb85d384f8f1e469d6013b1f9b5,3,"3 messed up ride into crazy town - stars what. the. hell. that was pretty much what i kept saying to myself as i read this book. it was definitely unlike anything else i have ever read. so...there's that. but considering i have spent the majority of the last 24 hours reading this book and the sequel/companion/continuation-of-the-fuckery, pull, i am not exactly sure if i truly enjoyed the story....or was just so wrapped up in how insane it was that i had to see how it all played out. annnnnyway,... book blurb: i feel like i am wrapped in a cyclone. everything is whirling around me, drawing the air out of my lungs and filling me with the best kind of turmoil. every time his tongue slides against mine, a prickle in my gut tells me how right we are together. how much i need david. how much i need us. i hope the cyclone never stops. emma searfoss has spent a lifetime trying to escape her abusive stepfather. it's why she moved far away from home. it's why she's kept no ties with her remaining family. and it's why she's got a major rage problem. when her neighbor shows up to fix the kitchen in her new apartment, his enigmatic charm calms the fire in her. david is cool and collected, and he makes emma feel safe for the first time ever. but david has his own chilling past--his six previous girlfriends have all disappeared without a trace. emma's walking a dangerous line, but david's pull is intoxicating. and impossible to resist... this is one of those books it is best to go into completely blind. the description doesn't really give anything away, which is a good thing. because trust me, if you fall into this story the way that i did, you will constantly be questioning what the hell is going on, as well as the sanity of the main characters. the book gets off to a crazy start from page one. i seriously challenge anyone to read the duel pov prologues and not have to find out how this story ends. so, i was obviously hooked immediately. and in the beginning, i actually thought that emma was more messed up than david. her mood swings can give anyone whiplash. and then i got more into the story....and further into david's flashbacks....and i realized that these two were in a fucked up league of their own. ""i want you to jump for me. jump off this bridge and let me save you. then i will believe that you love me, and i will love you back."" there several times when i wanted to just give up on this book. it was making my head spin, and confused me and made me want to yell at my kindle. but in the end...i just had to know. i had to know what the hell was going on. i had to know the motivations behind the things david did. and just when i thought i was going to get some answers, the book ends. on a ginormous cliffhanger! like, there could be no bigger cliffhanger than the way this book ends. and once again, i was angry and frustrated and wanted to throw kittens. and then it hit me. this author is a flipping genius. because her little messed up story completely took over my life while i was reading it. and while it was driving me nuts and i wasn't overly fond of the main characters....i still cared enough to keep reading. so well played ms wallis. well played.",eng,3,Wed Nov 12 05:34:06 -0800 2014,599,4.028368794326241,141
6344423,0000fbf05c6d154f1ae4362b09e02867,"Intertwined (Intertwined, #1)",48192,Gena Showalter,348a6f7145ef4593d511af0be45d1fa0,4,"this was an awesome book, i loved the concept of having souls in your head and i thought aden was awesome, sometimes i wanted to kill some characters, but the usual death threats. i actually liked a lot of the character... especially the souls: caleb was my fav, the little pervert.",eng,0,Thu Apr 11 12:04:03 -0700 2013,51,3.670138888888889,288
1

In [0]:
# Save the features dataset safely (no external location needed)
# This writes it as a managed Delta table inside Databricks

spark.sql("USE SCHEMA default")

(
    features
    .write
    .format("delta")
    .mode("overwrite")        # overwrite old version if rerun
    .option("overwriteSchema", "true")
    .saveAsTable("default.features_v1")
)

print("Successfully saved features_v1 as a managed Delta table in Databricks.")


Successfully saved features_v1 as a managed Delta table in Databricks.


In [0]:
# Step 5: Verify the Gold dataset (features_v1)

# Reload the saved table
spark.sql("USE SCHEMA default")
final_df = spark.table("features_v1")

# --- Basic info ---
print("Verification Results:")
print(f"Total Records: {final_df.count()}")
final_df.printSchema()

# --- Preview sample rows ---
display(final_df.limit(10))

# --- Basic descriptive statistics ---
final_df.select(
    "rating",
    "review_length_words",
    "avg_rating",
    "num_reviews"
).describe().show()


Verification Results:
Total Records: 14926784
root
 |-- book_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- avg_rating: double (nullable = true)
 |-- num_reviews: long (nullable = true)



book_id,review_id,title,author_id,name,user_id,rating,review_text,language_code,n_votes,date_added,review_length_words,avg_rating,num_reviews
2909013,00ab87e137e4c5b451153494ce2a7084,Vanish,1259390,Tom Pawlik,61d47f7eb4c29a1aea39ebe3451c80ec,2,whaaaat did i just read,eng,0,Wed Jul 08 03:31:32 -0700 2015,5,3.5454545454545454,22
32703721,0121e1c652ad65d5a84821b00adeb016,Not So Little Green Man,1709139,Celia Kyle,152890aa867aa36cecd028803ce0af21,4,"so far one of the best books in this series. i liked the characters, and the author just sucks you right in and off you go on an adventure. i really enjoy reading this author.",eng,0,Thu Oct 20 04:54:05 -0700 2016,35,4.230769230769231,26
13539044,0176b401e0bb9c97b743f48caa43b44c,The Silver Linings Playbook,1251730,Matthew Quick,61f95deafc169136765bd7eea6a1d538,5,"great story! i loved the way it was written, the expressions that were used, just all of it!",eng,0,Sat Apr 13 05:58:51 -0700 2013,18,4.002244668911335,1782
23480844,030987d1c85083e5f43c942b00f6a15d,Firsts,9816754,Laurie Elizabeth Flynn,6cd0b96677e585f2c0f60a89a5c078a8,4,"i couldn't put this book down. i read it in one sitting. the concept really intrigued me. the idea of a teenage girl giving sex ed to virgin boys was a new storyline. not being a fan of cheating, i didn't like that the boys had girlfriends. so, i thought i might not like the main character. but, i did end up liking her and rooting for her. it was sweet in a way but turned dark a few times. i anticipated the dark twists because sleeping with virtual strangers can be scary. i wish there was a non-prostitution way that virgins could work out their anxiety and fears.",eng,0,Thu Jul 07 21:57:54 -0700 2016,109,3.8419540229885056,348
63697,03e595b62b43c96dd9c628fac1148fd4,The Man Who Mistook His Wife for a Hat and Other Clinical Tales,843200,Oliver Sacks,867d8765c8c0a859befec4b1ff5638ed,4,"the content of this book deserves five stars, but the writing style does drag it down a little bit for me. it falls a little bit in between amusing non-fiction and an academic descriptive text based on qualitative story telling techniques. yet - a very important contribution to what various brain injuries and conditions can do to our personality and daily life, a basic must-read for anybody interested in neuropsychology.",en-US,0,Sun Jan 24 17:08:21 -0800 2016,70,3.7316239316239317,585
34020291,051352bd4c81e7d36c1899627d989af7,"Hurt Road: The Music, the Memories, and the Miles Between",17134633,Mark Lee,dcf821f915941e0702883b0e5ec24304,5,when i found out i had a chance to read and review mark lee's book hurt road i was super excited. i am a fan of third day and have loved their music for quite some time. reading about mark's life over the years was eye opening. i connected with this book on multiple levels. as a christian learning about his ups and downs in life along with some trails and tribulations was inspiring. as a fan i was intrigued to find out how the band came together. as a musician i have a new found fire to play for christ. there are parts in the book that brought tears to my eyes. like when i read about some of the tragedy that his family had to endure. the pages are a reminder that no matter who you are there are tough times in life but god will help us through. i gave this book 5 stars and would give it more if i could. i highly recommend this book.,,0,Wed Oct 04 13:42:54 -0700 2017,170,4.6,5
9464733,054e58b497eaab25153ae73ef0419700,Beauty Queens,2526,Libba Bray,c82ccd4944e12c89545a564696c26d61,5,"5 stars to the audiobook as well (read by libba herself) ok, i don't know where to begin. well, i guess i should start with...this was an amazing audiobook! libba did a fantastic job with the characters and it just added so much to the story. here's the gist of the story: a plane holding miss teen beauty contestants crashes on a desert island. this book is filled with tongue-in-cheek cliche's of beauty queens. but as the story continues, we get a really good look into eac

+-------+------------------+-------------------+------------------+------------------+
|summary|            rating|review_length_words|        avg_rating|       num_reviews|
+-------+------------------+-------------------+------------------+------------------+
|  count|          14926784|           14926784|          14926784|          14926784|
|   mean|3.9090020998495056| 130.18835852384547| 3.909002099849741| 626.6382319192131|
| stddev| 1.048491199413865|  181.6629259039522|0.5386594198035569|1761.3517460165488|
|    min|                 1|                  1|               1.0|                 1|
|    max|                 5|               5443|               5.0|             19990|
+-------+------------------+-------------------+------------------+------------------+

